In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

In [2]:
import sys
import os

new_path = "/home/drug_cnn/inception"
if new_path not in sys.path:
    sys.path.insert(0, new_path)

In [3]:
from inception import image_processing
from inception import inception_model as inception

In [4]:
#load label list
labels_file = "/home/drug_cnn/flowers-data/raw-data/labels.txt"
unique_labels = [l.strip() for l in tf.gfile.FastGFile(
      labels_file, 'r').readlines()]
unique_labels.insert(0, "Unknown")

In [5]:
unique_labels

['Unknown',
 'cannabis',
 'cocaine',
 'daisy',
 'dandelion',
 'ecstacy',
 'mushroom',
 'poppy',
 'roses',
 'sunflowers',
 'tulips']

In [6]:
from inception.data import build_image_data as image_bd

image_name = "/home/drug_cnn/test images/distractor9.jpg"
#image_name = "/home/drug_cnn/flowers-data/raw-data/train/sunflowers/6953297_8576bf4ea3.jpg"
#image_name = "/home/drug_cnn/flowers-data/raw-data/train/cocaine/920224_1280x720.jpg"

tensor_batch_size = 1
tensor_height = 299  #FLAGS.image_size
tensor_width = 299   # FLAGS.image_size
tensor_depth = 3
num_classes = 11

image_data, image_h,image_w = image_bd._process_image(image_name, image_bd.ImageCoder())

image_tensor = image_processing.eval_image(image_processing.decode_jpeg(image_data), tensor_height, tensor_width)
image_tensor = tf.sub(image_tensor, 0.5)
image_tensor = tf.mul(image_tensor, 2.0)

image_tensor = tf.cast(image_tensor, tf.float32)
image_tensor = tf.reshape(image_tensor, shape=[tensor_batch_size, tensor_height, tensor_width, tensor_depth])

In [7]:
# Restore the moving average version of the learned variables for eval.
logits, _ = inception.inference(image_tensor, num_classes, restore_logits=False)
variable_averages = tf.train.ExponentialMovingAverage(inception.MOVING_AVERAGE_DECAY)

saver = tf.train.Saver(variable_averages.variables_to_restore())

In [8]:
#import checkpointing
checkpoint_dir = "/home/drug_cnn/inception/model_test"

sess = tf.Session()

ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
if ckpt and ckpt.model_checkpoint_path:
    if os.path.isabs(ckpt.model_checkpoint_path):
        # Restores from checkpoint with absolute path.
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        # Restores from checkpoint with relative path.
        saver.restore(sess, os.path.join(checkpoint_dir,
                                         ckpt.model_checkpoint_path))

    # Assuming model_checkpoint_path looks something like:
    #   /my-favorite-path/imagenet_train/model.ckpt-0,
    # extract global_step from it.
    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
    print('Succesfully loaded model from %s at step=%s.' %
           (ckpt.model_checkpoint_path, global_step))
else:
    print('No checkpoint file found')

Succesfully loaded model from /home/drug_cnn/inception/model_test/model.ckpt-20000 at step=20000.


In [9]:
top_op = tf.nn.top_k(logits, k=5)
result = sess.run(top_op)

In [10]:
score_type = result[0].tolist()[0]
class_type = result[1].tolist()[0]

for i in range(0, 5):
    print(unique_labels[class_type[i]], score_type[i])

mushroom 3.38813614845
poppy 1.70150446892
cocaine -0.0822424888611
ecstacy -0.377949774265
Unknown -0.380021482706
